In [96]:
from bs4 import BeautifulSoup
import requests
import pandas 
import re
import time
import csv

Confingurando strings padrões:

In [164]:
def GetHTML(page_link):
    page_response = requests.get(page_link, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    return page_content

In [182]:
def gerarListaConsultas(subbase,ano):
    base = "http://sis.montesclaros.mg.gov.br"
    fim_hash = ['31%2F01%2F201','28%2F02%2F201','30%2F03%2F201','31%2F04%2F201',\
            '30%2F05%2F201','31%2F06%2F201','30%2F07%2F201','31%2F08%2F201',\
            '30%2F09%2F201','31%2F10%2F201','30%2F11%2F201','31%2F12%2F201']
    inicio_hash = ['01%2F01%2F201','01%2F02%2F201','01%2F03%2F201','01%2F04%2F201',\
            '01%2F05%2F201','01%2F06%2F201','01%2F07%2F201','31%2F08%2F201',\
            '01%2F09%2F201','01%2F10%2F201','01%2F11%2F201','31%2F12%2F201']
    
    
    lista_links = []
    for inicio,fim in zip(inicio_hash, fim_hash):
        inicio = "&inicio="+inicio+ano
        fim = "&fim="+fim+ano
        pagina = "&page=1"
        link = base+subbase+inicio+fim+pagina
        page_content = GetHTML(link)
        try:
            last_page = int(page_content.find_all('li')[-1].find('a')['href'].split("=")[-1])
            pagina = "&page="
            for i in range(1,last_page+1):
                link = base+subbase+inicio+fim+pagina+str(i)
                lista_links.append(link)
        except:
            pass

In [93]:
textContent = []
tuplas = []
context = page_content.find_all("td")
for coluna in context:
    valor = coluna.text
    if valor == '\n ':
        valor = coluna.find("a")['href']
        page_link_details = base+valor
        page_details_response = requests.get(page_link_details, timeout=5)
        page_details_content = BeautifulSoup(page_details_response.content, "html.parser")
        context_details = page_details_content.find_all("td")
        tam = len(context_details)
        for i in range(tam):
            if context_details[i].find('strong') != None :
                valor = context_details[i+1].text.strip()
                textContent.append(valor)

        tuplas.append(textContent)
        textContent = []
    else:
        textContent.append(valor.strip())

In [132]:
csvfile = open(arquivoOutput, 'a+', encoding='utf-8-sig') 
spamwriter = csv.writer(csvfile, delimiter=';', quoting=csv.QUOTE_MINIMAL)
# FAZER COLUNAS spamwriter.writerow(['Marca', 'Modelo', 'Ano'])
for tupla in tuplas:
    spamwriter.writerow(tupla)

In [161]:
last_page

76